In [1]:
import keras
from random import randint
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras.layers import Dense, Activation, Embedding, LSTM, Masking
from keras.optimizers import SGD
import numpy as np

Using TensorFlow backend.


ImportError: libcudart.so.7.5: cannot open shared object file: No such file or directory

In [2]:
maxlen = 80 # cut texts after this number of words (among top max_features most common words)
max_features = 200 # Anzahl der Wortarten
batch_size=128 # "Sätze"
RESULTS={
        '.': 1,
        ',': 2,
        '!': 3,
        '?': 4,
    }

In [3]:
dataset = open("dataset.txt")
all_data = list(map(lambda l: l.split(" "), dataset.readlines()))
all_words = [x for x in all_data if x][0] # remove empty tuples

In [4]:
model = Sequential()

model.add(Masking(mask_value=0, input_shape=(maxlen, max_features)))
#model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(len(RESULTS)+1))
#model.add(Activation('relu'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',  metrics=['accuracy'])

In [5]:
def last_char_to_result(last_char):
    i = RESULTS.get(last_char, 0)
    result = [0]*(len(RESULTS)+1)
    result[i] = 1
    return result

def clean(string):
    return string.replace(".", "").replace(",", "").replace("!", "").replace("?", "")

def load_sample(skip= True):
    sentence=[] # ['word','word',...]
    result=[] # [0,1,0,0,0] #6
    offset = randint(0, max(len(all_words), len(all_words) - maxlen - 2))
    num_words = randint(2, maxlen)
    sentence = all_words[offset:(offset+num_words)]
    if sentence == []:
        return load_sample()
    last_char = sentence[-1][-1]
    result = last_char_to_result(last_char)
    if (skip and (RESULTS.get(last_char, 0) == 0) and (randint(0,100)>10)):
        return load_sample()
    sentence = list(map(clean, sentence))
    return (" ".join(sentence),result)

def transform(sentence):
    x = text.one_hot(sentence, max_features, lower=True)
    x_new = [ [0 if t != i else 1 for i in range(max_features)] for t in x]
    for i in range(maxlen-len(x)):
        x_new.append( [0]*200 )
    return x_new

def create_dataset(size, skip= True):
    X=[];y=[]
    for iteration in range(0, size):
        sentence, result = load_sample(skip)
        X.append(transform(sentence)) 
        y.append(result)
    return np.array(X), np.array(y)

In [6]:
X, y = create_dataset(300)

model.fit(X, y, nb_epoch=40);

Exception: Error when checking model input: expected masking_input_1 to have 3 dimensions, but got array with shape (300, 1)

In [ ]:
X_test, y_test = create_dataset(10, False)
score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
satz,r = load_sample()
y = model.predict(np.array([transform(satz)]))[-1]

inv_results = {v: k for k, v in RESULTS.items()}
i=0
max_pos = np.argmax(y)
for prob in y:
    char = inv_results.get(i, ' ')
    ok = '*' if r[i]==1 else ''
    ok = ok + ("<" if max_pos == i else '')
    print(char + " = {:10.2f} ".format(prob*100) + (ok))
    i = i+1
    
    
print(satz)